# SQLAlchemy - Object Relational Mapping
## Creating Tables

In [1]:
import sqlalchemy
from sqlalchemy import create_engine, ForeignKey
from sqlalchemy import Column, Date, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref

print sqlalchemy.__version__

engine = create_engine('sqlite:///mymusic.db', echo=True)
Base = declarative_base()

class Artist(Base):
    """"""
    __tablename__ = 'artists'

    id = Column(Integer, primary_key=True)
    name = Column(String)

    def __init__(self, name):
        """"""
        self.name = name

class Album(Base):
    """"""
    __tablename__ = 'albums'
    id = Column(Integer, primary_key = True)
    title = Column(String)
    release_date = Column(Date)
    publisher = Column(String)
    media_type = Column(String)

    artist_id = Column(Integer, ForeignKey("artists.id"))
    artist = relationship("Artist", backref=backref("albums", order_by=id))

    def __init__(self, title, release_date, publisher, media_type):
        """"""
        self.title = title
        self.release_date = release_date
        self.publisher = publisher
        self.media_type = media_type

Base.metadata.create_all(engine)


0.8.2
2013-10-16 10:01:14,790 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("artists")
2013-10-16 10:01:14,790 INFO sqlalchemy.engine.base.Engine ()
2013-10-16 10:01:14,792 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("albums")
2013-10-16 10:01:14,792 INFO sqlalchemy.engine.base.Engine ()
2013-10-16 10:01:14,793 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE artists (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	PRIMARY KEY (id)
)


2013-10-16 10:01:14,795 INFO sqlalchemy.engine.base.Engine ()
2013-10-16 10:01:14,901 INFO sqlalchemy.engine.base.Engine COMMIT
2013-10-16 10:01:14,903 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE albums (
	id INTEGER NOT NULL, 
	title VARCHAR, 
	release_date DATE, 
	publisher VARCHAR, 
	media_type VARCHAR, 
	artist_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(artist_id) REFERENCES artists (id)
)


2013-10-16 10:01:14,904 INFO sqlalchemy.engine.base.Engine ()
2013-10-16 10:01:15,081 INFO sqlalchemy.engine.base.Engine COMMIT


## Inserting Records into Tables

In [2]:
import datetime
from sqlalchemy.orm import sessionmaker
#from table_def import Artist, Album

Session = sessionmaker(bind=engine)
session = Session()

new_artist = Artist("News Boys")
new_artist.albums = [Album("Read All About It", datetime.date(1988,12,1), "Refuge", "CD")]

more_albums = [Album("Hell Is For Wimps", datetime.date(1990,7,31), "Star Song", "CD"),
               Album("Love Liberty Disco", datetime.date(1999,11,16), "Sparrow", "CD"),
               Album("Thrive", datetime.date(2002,3,26), "Sparrow", "CD")]
new_artist.albums.extend(more_albums)

session.add(new_artist)
session.commit()

session.add_all([Artist("MXPX"), Artist("Kutless"), Artist("Thousand Foot Krutch")])
session.commit()

2013-10-16 10:01:15,098 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2013-10-16 10:01:15,099 INFO sqlalchemy.engine.base.Engine INSERT INTO artists (name) VALUES (?)
2013-10-16 10:01:15,101 INFO sqlalchemy.engine.base.Engine ('News Boys',)
2013-10-16 10:01:15,102 INFO sqlalchemy.engine.base.Engine INSERT INTO albums (title, release_date, publisher, media_type, artist_id) VALUES (?, ?, ?, ?, ?)
2013-10-16 10:01:15,104 INFO sqlalchemy.engine.base.Engine ('Read All ABout It', '1988-12-01', 'Refuge', 'CD', 1)
2013-10-16 10:01:15,104 INFO sqlalchemy.engine.base.Engine INSERT INTO albums (title, release_date, publisher, media_type, artist_id) VALUES (?, ?, ?, ?, ?)
2013-10-16 10:01:15,105 INFO sqlalchemy.engine.base.Engine ('Hell Is For Wimps', '1990-07-31', 'Star Song', 'CD', 1)
2013-10-16 10:01:15,105 INFO sqlalchemy.engine.base.Engine INSERT INTO albums (title, release_date, publisher, media_type, artist_id) VALUES (?, ?, ?, ?, ?)
2013-10-16 10:01:15,105 INFO sqlalchemy.engine.base

## Selecting Data from Tables

In [3]:
res = session.query(Artist).all()
for artist in res:
    print artist.name

res = session.query(Artist).filter(Artist.name=="News Boys").first()
print res.name

res  =session.query(Album).order_by(Album.title).all()
for album in res:
    print album.title

res = session.query(Artist, Album)
res = res.filter(Artist.id == Album.artist_id)
artist, album = res.filter(Album.title == "Thrive").first()
print artist.name, ':', album.title

res = session.query(Album).filter(Album.publisher.like("S%a%")).all()
for item in res:
    print item.publisher

2013-10-16 10:01:15,614 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2013-10-16 10:01:15,615 INFO sqlalchemy.engine.base.Engine SELECT artists.id AS artists_id, artists.name AS artists_name 
FROM artists
2013-10-16 10:01:15,615 INFO sqlalchemy.engine.base.Engine ()
News Boys
MXPX
Kutless
Thousand Foot Krutch
2013-10-16 10:01:15,618 INFO sqlalchemy.engine.base.Engine SELECT artists.id AS artists_id, artists.name AS artists_name 
FROM artists 
WHERE artists.name = ?
 LIMIT ? OFFSET ?
2013-10-16 10:01:15,618 INFO sqlalchemy.engine.base.Engine ('News Boys', 1, 0)
News Boys
2013-10-16 10:01:15,621 INFO sqlalchemy.engine.base.Engine SELECT albums.id AS albums_id, albums.title AS albums_title, albums.release_date AS albums_release_date, albums.publisher AS albums_publisher, albums.media_type AS albums_media_type, albums.artist_id AS albums_artist_id 
FROM albums ORDER BY albums.title
2013-10-16 10:01:15,621 INFO sqlalchemy.engine.base.Engine ()
Hell Is For Wimps
Love Liberty Disco
Read

## Modifying and Deleting Records

In [8]:
res = session.query(Artist).filter(Artist.name == "Kutless").first()
if res:
    print res.name
    res.name = "Beach Boys"
    session.commit()

artist, album = session.query(Artist, Album).filter(Artist.id==Album.artist_id).filter(Album.title=="Thrive").first()
album.title = "Step Up To The Microphone"
session.commit()

res = session.query(Artist, Album).filter(Artist.id==Album.artist_id).all()
for item in res:
    artist, album = item
    print artist.name, album.title

2013-10-16 10:05:26,780 INFO sqlalchemy.engine.base.Engine SELECT artists.id AS artists_id, artists.name AS artists_name 
FROM artists 
WHERE artists.name = ?
 LIMIT ? OFFSET ?
2013-10-16 10:05:26,782 INFO sqlalchemy.engine.base.Engine ('Kutless', 1, 0)
2013-10-16 10:05:26,783 INFO sqlalchemy.engine.base.Engine SELECT artists.id AS artists_id, artists.name AS artists_name, albums.id AS albums_id, albums.title AS albums_title, albums.release_date AS albums_release_date, albums.publisher AS albums_publisher, albums.media_type AS albums_media_type, albums.artist_id AS albums_artist_id 
FROM artists, albums 
WHERE artists.id = albums.artist_id AND albums.title = ?
 LIMIT ? OFFSET ?
2013-10-16 10:05:26,785 INFO sqlalchemy.engine.base.Engine ('Thrive', 1, 0)


TypeError: 'NoneType' object is not iterable